In [2]:
import zipfile as zp
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression 
import ipyparallel as ipp
import gc

In [3]:
df = pd.read_csv('train.zip', index_col = 0)
df

,visits
id,
1,30 84 126 135 137 179 242 342 426 456 460 462...
2,24 53 75 134 158 192 194 211 213 238 251 305 ...
3,51 143 173 257 446 491 504 510 559 616 719 73...
4,1 20 22 92 124 149 211 335 387 390 406 460 48...
5,34 51 56 106 110 121 163 233 266 275 345 359 ...
...,...
299996,6 12 15 30 33 48 49 50 55 56 70 74 76 83 84 9...
299997,4 7 12 18 19 35 41 53 55 60 75 77 81 82 91 95...
299998,1 9 15 17 29 31 42 49 62 74 85 105 109 113 11...


In [3]:
id_1_visit = pd.DataFrame({'Days': df.iloc[0].str.split(' ')[0]}).iloc[1:].astype(int)
id_1_visit.head()

,Days
1,30
2,84
3,126
4,135
5,137


In [4]:
id_1_visit.dtypes

Days    int32
dtype: object

In [5]:
id_1_visit.Days = id_1_visit.Days % 7 + 1

In [6]:
id_1_visit.head()

,Days
1,3
2,1
3,1
4,3
5,5


In [7]:

id_clear = id_1_visit.groupby(['Days']).agg({'Days' :'count'}).rename(columns={"Days":"Count"})
id_clear

,Count
Days,
1,5
2,2
3,7
4,2
5,7
6,2
7,4


In [8]:
id_clear['Density'] = id_clear['Count'] / id_clear['Count'].sum()

In [9]:
id_clear

,Count,Density
Days,,
1,5,0.172414
2,2,0.068966
3,7,0.241379
4,2,0.068966
5,7,0.241379
6,2,0.068966
7,4,0.137931


In [10]:
id_clear.Density.argmax(axis = -1) + 1

3

# Обработка всего сета

In [4]:
client = ipp.Client()
print(len(client), 'cores')

8 cores


In [5]:
lbview = client.load_balanced_view()

In [6]:
solution = pd.DataFrame({'id':[], 'nextvisit':[]})
solution

,id,nextvisit


In [7]:
def Predict(df):
    import pandas as pd
    id_i_visit = pd.DataFrame({'Days': df.str.split(' ')[0]}).iloc[1:].astype(int)
    id_i_visit.Days = id_i_visit.Days % 7 + 1
    return id_i_visit.Days.value_counts(normalize = True).argmax(axis = -1)

In [ ]:
n = df.shape[0]
res = {}
for i in range(n):
    _async = lbview.apply(Predict, df.iloc[i])
    ## store results
    res[i] = _async
client.wait()

In [ ]:
for i in range(n):
    solution.loc[i] = int(i + 1), int(res[i].get())

In [ ]:
solution = solution.astype(int)

In [ ]:
solution = solution.set_index(['id'])
solution

In [ ]:
solution.to_csv('submission.csv')

In [342]:
# id_merg = pd.DataFrame({'visit' : 0}, index = np.arange(1, 1100, 1))
# id_merg

In [343]:
# id_merg.loc[id_1_visit.index.astype(int).tolist(), 'visit'] = 1
# id_merg.describe()


In [344]:
# id_merg.visit.unique()

In [345]:
# id_merg.visit.value_counts()

In [346]:
# dumm = pd.get_dummies(id_merg.visit)
# dumm

In [347]:
# id_merg = id_merg.join(dumm)


In [348]:
# id_merg

In [349]:
# # X = np.array(id_merg.index.tolist()).reshape(1, -1)
# # Y = id_merg.iloc[:, 1:]
# # X

# X = np.array(id_merg.index).reshape(-1, 1)
# Y = id_merg.iloc[:, 0:]
# X

In [350]:
# clf = LogisticRegression().fit(X, np.ravel(Y))

In [351]:
# X = np.arange(1100, 3000, 1).reshape(-1, 1)
# pred = clf.predict(X)